In [1]:
!pip install -q transformers datasets accelerate evaluate scikit-learn pandas numpy matplotlib torch torchvision


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 13.9 MB/s eta 0:00:00


In [4]:
!pip install -U datasets huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 21.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.36.0
    Uninstalling huggingface-hub-0.36.0:
      Successfully uninstalled huggingface-hub-0.36.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.57.6 requires huggingface-hub<1.0,>=0.34.0, but you have hu

In [2]:
import time
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [4]:
import torch
print("GPU available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")


GPU available: True
GPU: Tesla T4


In [5]:
from datasets import load_dataset

dataset = load_dataset("stanfordnlp/imdb")


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [6]:
# speed hack for colab — use subset (optional but recommended)
dataset["train"] = dataset["train"].shuffle(seed=42).select(range(8000))
dataset["test"]  = dataset["test"].shuffle(seed=42).select(range(2000))


In [7]:
MODELS = [
    "bert-base-uncased",
    "roberta-base",
    "distilbert-base-uncased",
    "albert-base-v2",
    "xlnet-base-cased",
    "microsoft/deberta-v3-base"
]


In [8]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="macro"
    )
    acc = accuracy_score(labels, preds)

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }


In [ ]:
results = []

for model_name in MODELS:
    print(f"\n🚀 Training {model_name}")

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name, num_labels=2
    )

    def tokenize(batch):
        return tokenizer(
            batch["text"],
            truncation=True,
            padding="max_length",
            max_length=256
        )

    tokenized = dataset.map(tokenize, batched=True)
    tokenized.set_format("torch", columns=["input_ids", "attention_mask", "label"])

    args = TrainingArguments(
        output_dir="./tmp",
        save_strategy="no",
        logging_steps=50,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=1,
        fp16=torch.cuda.is_available(),
        report_to="none"
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=tokenized["train"],
        eval_dataset=tokenized["test"],
        compute_metrics=compute_metrics
    )

    trainer.train()

    # Evaluation
    metrics = trainer.evaluate()

    # Inference time (avg per sample)
    model.eval()
    inputs = tokenizer(
        dataset["test"][0]["text"],
        return_tensors="pt",
        truncation=True,
        padding=True
    ).to(model.device)

    start = time.time()
    with torch.no_grad():
        for _ in range(50):
            model(**inputs)
    infer_time = (time.time() - start) / 50 * 1000  # ms

    # Model size
    param_size = sum(p.numel() for p in model.parameters())
    model_size_mb = param_size * 4 / (4 ** 2)

    results.append({
        "model": model_name,
        "accuracy": metrics["eval_accuracy"],
        "f1": metrics["eval_f1"],
        "precision": metrics["eval_precision"],
        "recall": metrics["eval_recall"],
        "inference_time_ms": infer_time,
        "model_size_mb": model_size_mb
    })



🚀 Training bert-base-uncased


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Step,Training Loss
50,0.563400
100,0.408900
150,0.386900
200,0.355600
250,0.343100
300,0.324500
350,0.477000
400,0.342000
450,0.314900
500,0.310800


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [11]:
df = pd.DataFrame(results)
df


,model,accuracy,f1,precision,recall,inference_time_ms,model_size_mb
0,bert-base-uncased,0.8960,0.895900,0.897528,0.8960,14.287338,417.647469
1,roberta-base,0.9225,0.922490,0.922724,0.9225,10.926037,475.491219
2,distilbert-base-uncased,0.8955,0.895475,0.895880,0.8955,8.093052,255.413094
3,albert-base-v2,0.8480,0.847978,0.848201,0.8480,11.966295,44.575203
4,xlnet-base-cased,0.9165,0.916474,0.917011,0.9165,25.037289,447.503914
5,microsoft/deberta-v3-base,0.9365,0.936497,0.936574,0.9365,27.454996,703.520515


In [12]:
def topsis(df, weights, benefit_cols, cost_cols):
    data = df.copy()

    # Normalize
    for col in benefit_cols + cost_cols:
        data[col] = data[col] / np.sqrt((data[col] ** 2).sum())

    # Apply weights
    for col, w in weights.items():
        data[col] *= w

    # Ideal best / worst
    ideal_best = {}
    ideal_worst = {}

    for col in benefit_cols:
        ideal_best[col] = data[col].max()
        ideal_worst[col] = data[col].min()

    for col in cost_cols:
        ideal_best[col] = data[col].min()
        ideal_worst[col] = data[col].max()

    # Distances
    d_best = []
    d_worst = []

    for _, row in data.iterrows():
        d_best.append(np.sqrt(sum((row[col] - ideal_best[col]) ** 2 for col in ideal_best)))
        d_worst.append(np.sqrt(sum((row[col] - ideal_worst[col]) ** 2 for col in ideal_worst)))

    data["topsis_score"] = np.array(d_worst) / (np.array(d_best) + np.array(d_worst))
    data["rank"] = data["topsis_score"].rank(ascending=False)

    return data.sort_values("rank")


In [13]:
weights = {
    "accuracy": 0.25,
    "f1": 0.25,
    "precision": 0.15,
    "recall": 0.15,
    "inference_time_ms": 0.10,
    "model_size_mb": 0.10
}

benefit = ["accuracy", "f1", "precision", "recall"]
cost = ["inference_time_ms", "model_size_mb"]

topsis_df = topsis(df, weights, benefit, cost)
topsis_df


,model,accuracy,f1,precision,recall,inference_time_ms,model_size_mb,topsis_score,rank
3,albert-base-v2,0.095852,0.095852,0.057494,0.057511,0.027360,0.004133,0.790455,1.0
2,distilbert-base-uncased,0.101221,0.101221,0.060726,0.060732,0.018504,0.023684,0.745114,2.0
1,roberta-base,0.104272,0.104275,0.062545,0.062563,0.024982,0.044092,0.528557,3.0
0,bert-base-uncased,0.101277,0.101269,0.060837,0.060766,0.032667,0.038728,0.518775,4.0
4,xlnet-base-cased,0.103594,0.103595,0.062158,0.062157,0.057246,0.041497,0.337792,5.0
5,microsoft/deberta-v3-base,0.105855,0.105858,0.063484,0.063513,0.062774,0.065237,0.179392,6.0


In [16]:
import matplotlib.pyplot as plt

# Sort for clean ranking
plot_df = topsis_df.sort_values("topsis_score", ascending=True)

plt.figure(figsize=(10, 5), facecolor="#0f172a")
ax = plt.gca()
ax.set_facecolor("#0f172a")

bars = ax.barh(
    plot_df["model"],
    plot_df["topsis_score"]
)

# Text + labels
for bar in bars:
    width = bar.get_width()
    ax.text(
        width + 0.01,
        bar.get_y() + bar.get_height()/2,
        f"{width:.3f}",
        va="center",
        color="#e5e7eb",
        fontsize=9
    )

# Axes styling
ax.set_xlabel("TOPSIS Closeness Coefficient", color="#e5e7eb", fontsize=11)
ax.set_title("Model Ranking using TOPSIS", color="#f8fafc", fontsize=13, weight="bold")

ax.tick_params(colors="#e5e7eb")
for spine in ax.spines.values():
    spine.set_visible(False)

# Grid (subtle)
ax.grid(axis="x", linestyle="--", alpha=0.25)

plt.tight_layout()
plt.show()
